In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
def load_market_data(connector_name: str, trading_pair: str, data_type: str = "order_book") -> pd.DataFrame:
    """
    Load market data from files for a specific connector and trading pair.
    
    Args:
        connector_name: Name of the connector (e.g., "bitmart_paper_trade")
        trading_pair: Trading pair symbol (e.g., "LINK-USDT")
        data_type: Type of data to load ("order_book" or "trades")
    
    Returns:
        pd.DataFrame: Concatenated DataFrame containing all data from matching files
    """
    folder = root_path + f"/data/order_book/"
    
    # Define the pattern based on data type
    pattern = "order_book_snapshots" if data_type == "order_book" else "trades"
    
    # Find all matching files
    files = [
        file for file in os.listdir(folder) 
        if connector_name in file 
        and trading_pair in file 
        and pattern in file
    ]
    
    if not files:
        raise FileNotFoundError(f"No {data_type} files found for {connector_name} {trading_pair}")
    
    # Load and concatenate all matching files
    dfs = []
    for file in files:
        df = pd.read_json(folder + "/" + file, lines=True)
        dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

# Example usage:
order_book_df = load_market_data("bitmart_paper_trade", "LINK-USDT", "order_book")
trades_df = load_market_data("bitmart_paper_trade", "LINK-USDT", "trades")


In [ ]:
order_book_df

In [ ]:
exchanges = ["bitmart_paper_trade", "ascend_ex_paper_trade"]
trading_pair = "LINK-USDT"

# get all the order book files and trades files
# unpack the best bid and best ask for the columns bids and asks of the dataframe (is the first observation of the list that is a tuple (amount, price))
# plot with scatter the best bid and best ask over time
# plot with scatter the trades over time
# plot with scatter the mid price over time
# plot with scatter the bid ask spread over time

ob_data = {}
trades_data = {}

for exchange in exchanges:
    # Load the order book data
    ob = load_market_data(exchange, trading_pair, "order_book")
    ob["best_bid_price"] = ob["bids"].apply(lambda x: x[0][0])
    ob["best_ask_price"] = ob["asks"].apply(lambda x: x[0][0])
    ob["best_bid_amount"] = ob["bids"].apply(lambda x: x[0][1])
    ob["best_ask_amount"] = ob["asks"].apply(lambda x: x[0][1])
    ob.index = pd.to_datetime(ob["ts"], unit="s")
    ob_data[exchange] = ob
    # Load the trades data
    trades = load_market_data(exchange, trading_pair, "trades")
    trades["amount_quote"] = trades["q_base"] * trades["price"]
    trades.index = pd.to_datetime(trades["ts"], unit="s")
    trades_data[exchange] = trades


In [ ]:
trades_data["bitmart_paper_trade"]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_market_data(ob_df: pd.DataFrame, trades_df: pd.DataFrame, exchange: str, levels_to_plot: int = 10):
    """
    Create a plotly figure with order book and trades data.
    """
    fig = go.Figure()
    for level in range(1, levels_to_plot + 1):
        bids_price = ob_df["bids"].apply(lambda x: x[level][0])
        bids_amount = ob_df["bids"].apply(lambda x: x[level][1])
        asks_price = ob_df["asks"].apply(lambda x: x[level][0])
        asks_amount = ob_df["asks"].apply(lambda x: x[level][1])
        
        # Plot the bid line and add with color intensity based on amount

        fig.add_trace(
            go.Scatter(
                x=ob_df.index,
                y=bids_price,
                mode='lines',
                name=f'Bid {level}',
                line=dict(width=1),
                marker=dict(
                    size=6,
                    color=bids_amount,
                    colorscale='Reds',
                    showscale=True,
                    colorbar=dict(title='Bid Amount')
                )
            )
        )
        fig.add_trace(
            go.Scatter(
                x=ob_df.index,
                y=asks_price,
                mode='lines',
                name=f'Ask {level}',
                line=dict(width=1),
                marker=dict(
                    size=6,
                    color=asks_amount,
                    colorscale='Reds',
                    showscale=True,
                    colorbar=dict(title='Ask Amount')
                )
            )
        )

    
    # Trades trace remains unchanged
    fig.add_trace(
        go.Scatter(
            x=trades_df.index,
            y=trades_df['price'],
            mode='markers',
            name='Trades',
            marker=dict(
                # size=trades_df['amount_quote'] * 2,
                symbol='circle',
                color='blue',
                opacity=0.5
            )
        )
    )
    
    # Update layout
    fig.update_layout(
        title=f'Market Data for {exchange}',
        xaxis_title='Timestamp',
        yaxis_title='Price',
        showlegend=True
    )
    
    return fig
# Create plots for each exchange
for exchange in exchanges:
    fig = plot_market_data(
        ob_df=ob_data[exchange],
        trades_df=trades_data[exchange],
        exchange=exchange,
        levels_to_plot=1
    )
    fig.show()